# Phase 3: Feature Engineering for Business Signals

## 1. Business Problem Statement
Raw transaction data does not directly capture customer value, demand patterns,
or operational risk. To enable forecasting, segmentation, churn prediction, and
profit optimization, we must engineer features that represent meaningful business
behavior rather than individual transactions.

## 2. Why This Matters to the Business
Well-designed features allow the business to anticipate customer needs, forecast
demand accurately, identify high-value customers, and make proactive decisions.
Poor feature design leads to fragile models and misleading insights.

In [1]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)


In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/cleaned_data.csv")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
from src.feature_engineering import (
    add_time_features,
    add_profit_discount_features,
    create_customer_aggregates,
    create_rfm_features,
    build_feature_dataset
)

In [4]:
df = build_feature_dataset(df)
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Sales,Quantity,Discount,Profit,order_year,order_month,order_quarter,order_dayofweek,profit_margin,is_discounted
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,261.9600,2,0.00,41.9136,2016,11,4,1,0.1600,0
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,731.9400,3,0.00,219.5820,2016,11,4,1,0.3000,0
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,14.6200,2,0.00,6.8714,2016,6,2,6,0.4700,0
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,957.5775,5,0.45,-383.0310,2015,10,4,6,-0.4000,1
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,22.3680,2,0.20,2.5164,2015,10,4,6,0.1125,1


## FEATURE ENGINEERING THEMES

In [5]:
customer_agg_df = create_customer_aggregates(df)
customer_agg_df.head()

,Customer ID,total_sales,total_profit,avg_discount,num_orders
0,AA-10315,5563.560,-362.8825,0.090909,5
1,AA-10375,1056.390,277.3824,0.080000,9
2,AA-10480,1790.512,435.8274,0.016667,4
3,AA-10645,5086.935,857.8033,0.063889,6
4,AB-10015,886.156,129.3465,0.066667,3


In [6]:
rfm_df = create_rfm_features(df)
rfm_df.head()

,Customer ID,Recency,Frequency,Monetary
0,AA-10315,185,5,5563.560
1,AA-10375,20,9,1056.390
2,AA-10480,260,4,1790.512
3,AA-10645,56,6,5086.935
4,AB-10015,416,3,886.156


In [7]:
df.to_csv("../data/processed/featured_data.csv", index=False)
customer_agg_df.to_csv("../data/processed/customer_rfm.csv", index=False)

In [1]:
# ======================================================
# EXECUTIVE METRICS — SINGLE SOURCE OF TRUTH
# ======================================================

import pandas as pd
from pathlib import Path

# ------------------------------------------------------
# 0. LOAD FEATURED DATA (THIS WAS MISSING)
# ------------------------------------------------------
BASE_DIR = Path(".")
df = pd.read_csv("../data/processed/featured_data.csv",
    parse_dates=["Order Date"]
)

# ------------------------------------------------------
# 1. Monthly executive aggregation
# ------------------------------------------------------
monthly_exec_metrics = (
    df
    .assign(
        Order_Month=df["Order Date"]
        .dt.to_period("M")
        .dt.to_timestamp()
    )
    .groupby("Order_Month")
    .agg(
        total_sales=("Sales", "sum"),
        total_profit=("Profit", "sum"),
        avg_discount=("Discount", "mean"),
        total_quantity=("Quantity", "sum")
    )
    .reset_index()
)

# ------------------------------------------------------
# 2. Executive KPIs (REQUIRED BY STREAMLIT)
# ------------------------------------------------------

# Profit margin
monthly_exec_metrics["profit_margin"] = (
    monthly_exec_metrics["total_profit"] /
    monthly_exec_metrics["total_sales"]
)

# Month-over-Month growth
monthly_exec_metrics["sales_mom"] = (
    monthly_exec_metrics["total_sales"].pct_change()
)

monthly_exec_metrics["profit_mom"] = (
    monthly_exec_metrics["total_profit"].pct_change()
)

# Remove first month (no MoM baseline)
monthly_exec_metrics = (
    monthly_exec_metrics
    .dropna()
    .reset_index(drop=True)
)

# ------------------------------------------------------
# 3. SAVE (THIS FILE FEEDS STREAMLIT)
# ------------------------------------------------------
output_dir = Path("data/processed")
output_dir.mkdir(parents=True, exist_ok=True)


monthly_exec_metrics.to_csv("../data/processed/executive_metrics_monthly.csv", index=False)
print("✅ executive_metrics_monthly.csv regenerated correctly")
monthly_exec_metrics.head()

✅ executive_metrics_monthly.csv regenerated correctly


,Order_Month,total_sales,total_profit,avg_discount,total_quantity,profit_margin,sales_mom,profit_mom
0,2014-02-01,4519.8920,862.3084,0.176087,159,0.190781,-0.682523,-0.648065
1,2014-03-01,55691.0090,498.7299,0.167516,585,0.008955,11.321314,-0.421634
2,2014-04-01,28295.3450,3488.8352,0.110000,536,0.123301,-0.491923,5.995440
3,2014-05-01,23648.2870,2738.7096,0.155328,466,0.115810,-0.164234,-0.215007
4,2014-06-01,34595.1276,4976.5244,0.172000,521,0.143850,0.462902,0.817106


## 3. Executive Summary

This phase converted raw transactional data into business-level signals capturing
seasonality, customer value, profitability, and purchasing behavior. These features
serve as the foundation for forecasting, segmentation, churn prediction, and pricing
optimization models in subsequent phases.
